1. Learning sgRNA predictors from empirical data
    * Load scripts and empirical data
    * Generate TSS annotation using FANTOM dataset
    * Calculate parameters for empirical sgRNAs
    * Fit parameters
2. Applying machine learning model to predict sgRNA activity
    * Find all sgRNAs in genomic regions of interest 
    * Predicting sgRNA activity
3. Construct sgRNA libraries
    * Score sgRNAs for off-target potential
* Pick the top sgRNAs for a library, given predicted activity scores and off-target filtering
* Design negative controls matching the base composition of the library
* Finalizing library design

# 1. Learning sgRNA predictors from empirical data
## Load scripts and empirical data

In [1]:
import sys
sys.path.insert(0, '../ScreenProcessing/')
%run sgRNA_learning.py

In [2]:
genomeDict = loadGenomeAsDict('large_data_files/hg19.fa')

Loading genome file...Done


In [3]:
#to use pre-calculated sgRNA activity score data (e.g. provided CRISPRi training data), load the following:
#CRISPRa activity score data also included in data_files
libraryTable_training = pd.read_csv('data_files/CRISPRi_trainingdata_libraryTable.txt', sep='\t', index_col = 0)
libraryTable_training.head()

,sublibrary,gene,transcripts,sequence
sgId,,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,drug_targets+kinase_phosphatase,AARS,all,GCCCCAGGATCAGGCCCCGCG
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,drug_targets+kinase_phosphatase,AARS,all,GGCCGCCCTCGGAGAGCTCTG
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,drug_targets+kinase_phosphatase,AARS,all,GACGGCGACCCTAGGAGAGGT
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,drug_targets+kinase_phosphatase,AARS,all,GGTGCAGCGGGCCCTTGGCGG
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,drug_targets+kinase_phosphatase,AARS,all,GCGCTCTGATTGGACGGAGCG


In [4]:
sgInfoTable_training = pd.read_csv('data_files/CRISPRi_trainingdata_sgRNAInfoTable.txt', sep='\t', index_col=0)
sgInfoTable_training.head()

,Sublibrary,gene_name,length,pam coordinate,pass_score,position,strand,transcript_list
sgId,,,,,,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,Drug_Targets+Kinase_Phosphatase,AARS,24,70323216,e39m1,70323216,+,['all']
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,Drug_Targets+Kinase_Phosphatase,AARS,24,70323296,e39m1,70323296,+,['all']
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,Drug_Targets+Kinase_Phosphatase,AARS,24,70323318,e39m1,70323318,+,['all']
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,Drug_Targets+Kinase_Phosphatase,AARS,24,70323362,e39m1,70323362,+,['all']
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,Drug_Targets+Kinase_Phosphatase,AARS,24,70323441,e39m1,70323441,+,['all']


In [5]:
activityScores = pd.read_csv('data_files/CRISPRi_trainingdata_activityScores.txt',sep='\t',index_col=0, header=None).iloc[:,0]
activityScores.head()

0
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1    0.348892
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1    0.912409
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1    0.997242
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1    0.962154
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1    0.019320
Name: 1, dtype: float64

In [6]:
tssTable = pd.read_csv('data_files/human_tssTable.txt',sep='\t', index_col=range(2))
tssTable.head()

position strand  \
gene  transcripts                                                           
A1BG  all                                                 58864864      -   
A1CF  ENST00000373993.1                                   52619744      -   
      ENST00000374001.2,ENST00000373997.3,ENST0000028...  52645434      -   
A2M   all                                                  9268752      -   
A2ML1 all                                                  8975067      +   

                                                         chromosome  \
gene  transcripts                                                     
A1BG  all                                                     chr19   
A1CF  ENST00000373993.1                                       chr10   
      ENST00000374001.2,ENST00000373997.3,ENST0000028...      chr10   
A2M   all                                                     chr12   
A2ML1 all                                                     chr12   

                                                                                           cage peak ranges  
gene  transcripts                                                                                            
A1BG  all                                                      [(58864822, 58864847), (58864848, 58864868)]  
A1CF  ENST00000373993.1                                                                                  []  
      ENST00000374001.2,ENST00000373997.3,ENST0000028...       [(52645379, 52645393), (52645416, 52645444)]  
A2M   all                                                 [(9268547, 9268556), (9268559, 9268568), (9268...  
A2ML1 all                                                 [(8975061, 8975072), (8975101, 8975108), (8975...

In [7]:
p1p2Table = pd.read_csv('data_files/human_p1p2Table.txt',sep='\t', header=0, index_col=range(2))
p1p2Table['primary TSS'] = p1p2Table['primary TSS'].apply(lambda tupString: (int(tupString.strip('()').split(', ')[0].split('.')[0]), int(tupString.strip('()').split(', ')[1].split('.')[0])))
p1p2Table['secondary TSS'] = p1p2Table['secondary TSS'].apply(lambda tupString: (int(tupString.strip('()').split(', ')[0].split('.')[0]),int(tupString.strip('()').split(', ')[1].split('.')[0])))
p1p2Table.head()

chromosome strand           TSS source           primary TSS  \
gene  transcript                                                                
A1BG  P1              chr19      -  CAGE, matched peaks  (58858938, 58859039)   
      P2              chr19      -  CAGE, matched peaks  (58864822, 58864847)   
A1CF  P1P2            chr10      -  CAGE, matched peaks  (52645379, 52645393)   
A2M   P1P2            chr12      -  CAGE, matched peaks    (9268507, 9268523)   
A2ML1 P1P2            chr12      +  CAGE, matched peaks    (8975206, 8975223)   

                         secondary TSS  
gene  transcript                        
A1BG  P1          (58858938, 58859039)  
      P2          (58864822, 58864847)  
A1CF  P1P2        (52645379, 52645393)  
A2M   P1P2          (9268528, 9268542)  
A2ML1 P1P2          (8975144, 8975169)

## Calculate parameters for empirical sgRNAs

### Because scikit-learn currently does not support any robust method for saving and re-loading the machine learning model, the best strategy is to simply re-learn the model from the training data

In [8]:
#Load bigwig files for any chromatin data of interest
bwhandleDict = {'dnase':BigWigFile(open('large_data_files/wgEncodeOpenChromDnaseK562BaseOverlapSignalV2.bigWig')),
'faire':BigWigFile(open('large_data_files/wgEncodeOpenChromFaireK562Sig.bigWig')),
'mnase':BigWigFile(open('large_data_files/wgEncodeSydhNsomeK562Sig.bigWig'))}

In [9]:
paramTable_trainingGuides = generateTypicalParamTable(libraryTable_training,sgInfoTable_training, tssTable, p1p2Table, genomeDict, bwhandleDict)

....

/usr/local/lib/python2.7/dist-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/numpy/lib/nanfunctions.py:326: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


.Done!

## Fit parameters

In [10]:
#load in the 5-fold cross-validation splits used to generate the model
import cPickle
with open('data_files/CRISPRi_trainingdata_traintestsets.txt') as infile:
    geneFold_train, geneFold_test, fitTable = cPickle.load(infile)

In [11]:
transformedParams_train, estimators = fitParams(paramTable_trainingGuides.loc[activityScores.dropna().index].iloc[geneFold_train], activityScores.loc[activityScores.dropna().index].iloc[geneFold_train], fitTable)

transformedParams_test = transformParams(paramTable_trainingGuides.loc[activityScores.dropna().index].iloc[geneFold_test], fitTable, estimators)

reg = linear_model.ElasticNetCV(l1_ratio=[.5, .75, .9, .99,1], n_jobs=16, max_iter=2000)

scaler = preprocessing.StandardScaler()
reg.fit(scaler.fit_transform(transformedParams_train), activityScores.loc[activityScores.dropna().index].iloc[geneFold_train])
predictedScores = pd.Series(reg.predict(scaler.transform(transformedParams_test)), index=transformedParams_test.index)
testScores = activityScores.loc[activityScores.dropna().index].iloc[geneFold_test]

print 'Prediction AUC-ROC:', metrics.roc_auc_score((testScores >= .75).values, np.array(predictedScores.values,dtype='float64'))
print 'Prediction R^2:', reg.score(scaler.transform(transformedParams_test), testScores)
print 'Regression parameters:', reg.l1_ratio_, reg.alpha_
coefs = pd.DataFrame(zip(*[abs(reg.coef_),reg.coef_]), index = transformedParams_test.columns, columns=['abs','true'])
print 'Number of features used:', len(coefs) - sum(coefs['abs'] < .00000000001)

('distance', 'primary TSS-Up') {'C': 0.05, 'gamma': 0.0001}
('distance', 'primary TSS-Down') {'C': 0.5, 'gamma': 5e-05}
('distance', 'secondary TSS-Up') {'C': 0.1, 'gamma': 5e-05}
('distance', 'secondary TSS-Down') {'C': 0.1, 'gamma': 5e-05}


/home/mhorlbeck/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:323: UserWarning: StandardScaler assumes floating point values as input, got object
  "got %s" % (estimator, X.dtype))
/home/mhorlbeck/.local/lib/python2.7/site-packages/sklearn/linear_model/base.py:400: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if precompute == 'auto':


Prediction AUC-ROC: 0.803109696478
Prediction R^2: 0.31263687609
Regression parameters: 0.5 0.00534455043278
Number of features used: 327


In [ ]:
#can save state for reproducing estimators later
#the pickling of the scikit-learn estimators/regressors will allow the model to be reloaded for prediction of other guide designs, 
#   but will not be compatible across scikit-learn versions, so it is important to preserve the training data and training/test folds
import cPickle
estimatorString = cPickle.dumps((fitTable, estimators, scaler, reg))
with open(PICKLE_FILE,'w') as outfile:
    outfile.write(estimatorString)
    
#also save the transformed parameters as these can slightly differ based on the automated binning strategy
transformedParams_train.head().to_csv(TRANSFORMED_PARAM_HEADER,sep='\t')

# 2. Applying machine learning model to predict sgRNA activity

## Generate TSS annotation using FANTOM dataset

In [12]:
#you can supply any table of gene transcription start sites formatted as below
#for demonstration purposes, the rest of this walkthrough will use a small arbitrary subset of the protein coding TSS table
tssTable_new = tssTable.iloc[10:20, :-1]
tssTable_new.head()

position strand  \
gene    transcripts                                                            
AADACL2 all                                                 151451714      +   
AADAT   ENST00000337664.4                                   171011117      -   
        ENST00000337664.4,ENST00000509167.1,ENST0000035...  171011284      -   
        ENST00000509167.1,ENST00000515480.1,ENST0000035...  171011424      -   
AAED1   all                                                  99417562      -   

                                                           chromosome  
gene    transcripts                                                    
AADACL2 all                                                      chr3  
AADAT   ENST00000337664.4                                        chr4  
        ENST00000337664.4,ENST00000509167.1,ENST0000035...       chr4  
        ENST00000509167.1,ENST00000515480.1,ENST0000035...       chr4  
AAED1   all                                                      chr9

In [13]:
#if desired, use the ensembl annotation and the HGNC database to supply gene aliases to assist P1P2 matching in the next step
gencodeData = loadGencodeData('large_data_files/gencode.v19.annotation.gtf')
geneToAliases = generateAliasDict('large_data_files/20150424_HGNC_symbols.txt',gencodeData)

Loading annotation file...Done


In [14]:
#Now create a TSS annotation by searching for P1 and P2 peaks near annotated TSSs
#same parameters as for our lncRNA libraries
p1p2Table_new = generateTssTable_P1P2strategy(tssTable_new, 'large_data_files/TSS_human.sorted.bed.gz', 
                                                  matchedp1p2Window = 30000, #region around supplied TSS annotation to search for a FANTOM P1 or P2 peak that matches the gene name (or alias)
                                                  anyp1p2Window = 500, #region around supplied TSS annotation to search for the nearest P1 or P2 peak
                                                  anyPeakWindow = 200, #region around supplied TSS annotation to search for any CAGE peak
                                                  minDistanceForTwoTSS = 1000, #If a P1 and P2 peak are found, maximum distance at which to combine into a single annotation (with primary/secondary TSS positions)
                                                  aliasDict = geneToAliases[0])
#the function will report some collisions of IDs due to use of aliases and redundancy in genome, but will resolve these itself

In [15]:
p1p2Table_new.head()

,,chromosome,strand,TSS source,primary TSS,secondary TSS
gene,transcript,,,,,
AADACL2,P1P2,chr3,+,"CAGE, matched peaks","(151451707, 151451722)","(151451707, 151451722)"
AADAT,P1P2,chr4,-,"CAGE, matched peaks","(171011323, 171011408)","(171011084, 171011147)"
AAED1,P1P2,chr9,-,"CAGE, matched peaks","(99417562, 99417609)","(99417615, 99417622)"
AAGAB,P1P2,chr15,-,"CAGE, matched peaks","(67546963, 67547024)","(67546963, 67547024)"
AAK1,P1P2,chr2,-,"CAGE, matched peaks","(69870747, 69870812)","(69870854, 69870878)"


In [16]:
p1p2Table_new.groupby('TSS source').agg(len).iloc[:,[2]]

,primary TSS
TSS source,
"CAGE, matched peaks",8


In [17]:
len(p1p2Table_new)

8

In [ ]:
#save tables
tssTable_alllncs.to_csv(TSS_TABLE_PATH,sep='\t', index_col=range(2))
p1p2Table_alllncs.to_csv(P1P2_TABLE_PATH,sep='\t', header=0, index_col=range(2))

## Find all sgRNAs in genomic regions of interest 

In [18]:
libraryTable_new, sgInfoTable_new = findAllGuides(p1p2Table_new, genomeDict, 
                                                  (-25,500)) #region around P1P2 TSSs to search for new sgRNAs; recommend -550,-25 for CRISPRa

In [19]:
len(libraryTable_new)

1125

In [20]:
libraryTable_new.head()

,gene,transcripts,sequence,genomic sequence
sgId,,,,
AADACL2_+_151451720.23-P1P2,AADACL2,P1P2,GTAGACTTGGGAACTCTCTC,CCTGAGAGAGTTCCCAAGTCTAC
AADACL2_+_151451732.23-P1P2,AADACL2,P1P2,GGTAGAGCAATTGTAGACTT,CCCAAGTCTACAATTGCTCTACT
AADACL2_+_151451733.23-P1P2,AADACL2,P1P2,GAGTAGAGCAATTGTAGACT,CCAAGTCTACAATTGCTCTACTA
AADACL2_-_151451809.23-P1P2,AADACL2,P1P2,GCTCAGTACTGTGAAGAAGC,TCTCAGTACTGTGAAGAAGCTGG
AADACL2_-_151451816.23-P1P2,AADACL2,P1P2,GCTGTGAAGAAGCTGGAAAA,ACTGTGAAGAAGCTGGAAAAAGG


## Predicting sgRNA activity

In [21]:
#calculate parameters for new sgRNAs
paramTable_new = generateTypicalParamTable(libraryTable_new, sgInfoTable_new, tssTable_new, p1p2Table_new, genomeDict, bwhandleDict)

.....Done!

In [22]:
paramTable_new.head()

length       distance                   \
                            length primary TSS-Up primary TSS-Down   
sgId                                                                 
AADACL2_+_151451720.23-P1P2     20             13               -2   
AADACL2_+_151451732.23-P1P2     20             25               10   
AADACL2_+_151451733.23-P1P2     20             26               11   
AADACL2_-_151451809.23-P1P2     20            102               87   
AADACL2_-_151451816.23-P1P2     20            109               94   

                                                                homopolymers  \
                            secondary TSS-Up secondary TSS-Down            A   
sgId                                                                           
AADACL2_+_151451720.23-P1P2               13                 -2            2   
AADACL2_+_151451732.23-P1P2               25                 10            2   
AADACL2_+_151451733.23-P1P2               26                 11            2   
AADACL2_-_151451809.23-P1P2              102                 87            2   
AADACL2_-_151451816.23-P1P2              109                 94            4   

                                     base fractions  ...    \
                             G  C  T              A  ...     
sgId                                                 ...     
AADACL2_+_151451720.23-P1P2  3  1  2           0.20  ...     
AADACL2_+_151451732.23-P1P2  2  1  2           0.30  ...     
AADACL2_+_151451733.23-P1P2  1  1  2           0.35  ...     
AADACL2_-_151451809.23-P1P2  1  1  1           0.30  ...     
AADACL2_-_151451816.23-P1P2  2  1  1           0.40  ...     

                            RNA folding-pairing, no scaffold                \
                                                         -12    -11    -10   
sgId                                                                         
AADACL2_+_151451720.23-P1P2                             True   True  False   
AADACL2_+_151451732.23-P1P2                            False  False  False   
AADACL2_+_151451733.23-P1P2                            False  False  False   
AADACL2_-_151451809.23-P1P2                            False   True   True   
AADACL2_-_151451816.23-P1P2                             True   True   True   

                                                                              
                                -9     -8     -7     -6     -5     -4     -3  
sgId                                                                          
AADACL2_+_151451720.23-P1P2  False  False  False   True   True   True   True  
AADACL2_+_151451732.23-P1P2  False  False  False  False  False  False  False  
AADACL2_+_151451733.23-P1P2  False  False  False  False  False  False  False  
AADACL2_-_151451809.23-P1P2   True  False  False  False  False  False  False  
AADACL2_-_151451816.23-P1P2  False  False  False  False  False  False  False  

[5 rows x 808 columns]

In [ ]:
#if starting from a separate session from where you ran the sgRNA learning steps of Part 1, reload the following
import cPickle
with open(PICKLE_FILE) as infile:
    fitTable, estimators, scaler, reg = cPickle.load(infile)
    
transformedParams_train = pd.read_csv(TRANSFORMED_PARAM_HEADER,sep='\t')

In [23]:
#transform and predict scores according to sgRNA prediction model
transformedParams_new = transformParams(paramTable_new, fitTable, estimators)

#reconcile any differences in column headers generated by automated binning
colTups = []
for (l1, l2), col in transformedParams_new.iteritems():
    colTups.append((l1,str(l2)))
transformedParams_new.columns = pd.MultiIndex.from_tuples(colTups)

predictedScores_new = pd.Series(reg.predict(scaler.transform(transformedParams_new.loc[:, transformedParams_train.columns].fillna(0).values)), index=transformedParams_new.index)

In [24]:
predictedScores_new.head()

sgId
AADACL2_+_151451720.23-P1P2    0.641245
AADACL2_+_151451732.23-P1P2    0.693926
AADACL2_+_151451733.23-P1P2    0.655759
AADACL2_-_151451809.23-P1P2    0.500835
AADACL2_-_151451816.23-P1P2    0.434376
dtype: float64

In [ ]:
libraryTable_new.to_csv(LIBRARY_TABLE_PATH,sep='\t')
sgInfoTable_new.to_csv(sgRNA_INFO_PATH,sep='\t')
predictedScores_new.to_csv(PREDICTED_SCORES_PATH, sep='\t')

# 3. Construct sgRNA libraries
## Score sgRNAs for off-target potential

### There are many ways to score sgRNAs as off-target; below is one listed one method that is simple and flexible, but ignores gapped alignments, alternate PAMs, and uses bowtie which may not be maximally sensitive in all cases

In [27]:
!mkdir temp_bowtie_files

In [28]:
#output all sequences to a temporary FASTQ file for running bowtie alignment
fqFile = 'temp_bowtie_files/bowtie_input.fq'

def outputTempBowtieFastq(libraryTable, outputFileName):
    phredString = 'I4!=======44444+++++++' #weighting for how impactful mismatches are along sgRNA sequence 
    with open(outputFileName,'w') as outfile:
        for name, row in libraryTable.iterrows():
            outfile.write('@' + name + '\n')
            outfile.write('CCN' + str(Seq.Seq(row['sequence'][1:]).reverse_complement()) + '\n')
            outfile.write('+\n')
            outfile.write(phredString + '\n')
            
outputTempBowtieFastq(libraryTable_new, fqFile)

In [29]:
import subprocess

#specifying a list of parameters to run bowtie with
#each tuple contains
# *the mismatch threshold below which a site is considered a potential off-target (higher is more stringent)
# *the number of sites allowed (1 is minimum since each sgRNA should have one true site in genome)
# *the genome index against which to align the sgRNA sequences; these can be custom built to only consider sites near TSSs
# *a name for the bowtie run to create appropriately named output files
alignmentList = [(39,1,'large_data_files/hg19.ensemblTSSflank500b','39_nearTSS'),
                (31,1,'large_data_files/hg19.ensemblTSSflank500b','31_nearTSS'),
                (21,1,'large_data_files/hg19_maskChrMandPAR','21_genome'),
                (31,2,'large_data_files/hg19.ensemblTSSflank500b','31_2_nearTSS'),
                (31,3,'large_data_files/hg19.ensemblTSSflank500b','31_3_nearTSS')]

alignmentColumns = []
for btThreshold, mflag, bowtieIndex, runname in alignmentList:

    alignedFile = 'temp_bowtie_files/' + runname + '_aligned.txt'
    unalignedFile = 'temp_bowtie_files/' + runname + '_unaligned.fq'
    maxFile = 'temp_bowtie_files/' + runname + '_max.fq'
    
    bowtieString = 'bowtie -n 3 -l 15 -e '+str(btThreshold)+' -m ' + str(mflag) + ' --nomaqround -a --tryhard -p 16 --chunkmbs 256 ' + bowtieIndex + ' --suppress 5,6,7 --un ' + unalignedFile + ' --max ' + maxFile + ' '+ ' -q '+fqFile+' '+ alignedFile
    print bowtieString
    print subprocess.call(bowtieString, shell=True) #0 means finished without errors

    #parse through the file of sgRNAs that exceeded "m", the maximum allowable alignments, and mark "True" any that are found
    try:
        with open(maxFile) as infile:
            sgsAligning = set()
            for i, line in enumerate(infile):
                if i%4 == 0: #id line
                    sgsAligning.add(line.strip()[1:])
    except IOError: #no sgRNAs exceeded m, so no maxFile created
        sgsAligning = set()
                    
    alignmentColumns.append(libraryTable_new.apply(lambda row: row.name in sgsAligning, axis=1))
    
#collate results into a table, and flip the boolean values to yield the sgRNAs that passed filter as True
alignmentTable = pd.concat(alignmentColumns,axis=1, keys=zip(*alignmentList)[3]).ne(True)

bowtie -n 3 -l 15 -e 39 -m 1 --nomaqround -a --tryhard -p 16 --chunkmbs 256 large_data_files/hg19.ensemblTSSflank500b --suppress 5,6,7 --un temp_bowtie_files/39_nearTSS_unaligned.fq --max temp_bowtie_files/39_nearTSS_max.fq  -q temp_bowtie_files/bowtie_input.fq temp_bowtie_files/39_nearTSS_aligned.txt
0
bowtie -n 3 -l 15 -e 31 -m 1 --nomaqround -a --tryhard -p 16 --chunkmbs 256 large_data_files/hg19.ensemblTSSflank500b --suppress 5,6,7 --un temp_bowtie_files/31_nearTSS_unaligned.fq --max temp_bowtie_files/31_nearTSS_max.fq  -q temp_bowtie_files/bowtie_input.fq temp_bowtie_files/31_nearTSS_aligned.txt
0
bowtie -n 3 -l 15 -e 21 -m 1 --nomaqround -a --tryhard -p 16 --chunkmbs 256 large_data_files/hg19_maskChrMandPAR --suppress 5,6,7 --un temp_bowtie_files/21_genome_unaligned.fq --max temp_bowtie_files/21_genome_max.fq  -q temp_bowtie_files/bowtie_input.fq temp_bowtie_files/21_genome_aligned.txt
0
bowtie -n 3 -l 15 -e 31 -m 2 --nomaqround -a --tryhard -p 16 --chunkmbs 256 large_data_files/

In [30]:
alignmentTable.head() #True = passed threshold

,39_nearTSS,31_nearTSS,21_genome,31_2_nearTSS,31_3_nearTSS
sgId,,,,,
AADACL2_+_151451720.23-P1P2,True,True,False,True,True
AADACL2_+_151451732.23-P1P2,True,True,True,True,True
AADACL2_+_151451733.23-P1P2,True,True,True,True,True
AADACL2_-_151451809.23-P1P2,False,True,False,True,True
AADACL2_-_151451816.23-P1P2,True,True,False,True,True


## Pick the top sgRNAs for a library, given predicted activity scores and off-target filtering

In [31]:
#combine all generated data into one master table
predictedScores_new.name = 'predicted score'
v2Table = pd.concat((libraryTable_new, predictedScores_new, alignmentTable, sgInfoTable_new), axis=1, keys=['library table v2', 'predicted score', 'off-target filters', 'sgRNA info'])

In [32]:
import re
#for our pCRISPRi/a-v2 vector, we append flanking sequences to each sgRNA sequence for cloning and require the oligo to contain
#exactly 1 BstXI and BlpI site each for cloning, and exactly 0 SbfI sites for sequencing sample preparation
restrictionSites = {re.compile('CCA......TGG'):1,
                   re.compile('GCT.AGC'):1,
                   re.compile('CCTGCAGG'):0}

def matchREsites(sequence, REdict):
    seq = sequence.upper()
    for resite, numMatchesExpected in restrictionSites.iteritems():
        if len(resite.findall(seq)) != numMatchesExpected:
            return False
        
    return True

def checkOverlaps(leftPosition, acceptedLeftPositions, nonoverlapMin):
    for pos in acceptedLeftPositions:
        if abs(pos - leftPosition) < nonoverlapMin:
            return False
    return True

In [33]:
#flanking sequences
upstreamConstant = 'CCACCTTGTTG'
downstreamConstant = 'GTTTAAGAGCTAAGCTG'

#minimum overlap between two sgRNAs targeting the same TSS
nonoverlapMin = 3

#number of sgRNAs to pick per gene/TSS
sgRNAsToPick = 10

#list of off-target filter (or combinations of filters) levels, matching the names in the alignment table above
offTargetLevels = [['31_nearTSS', '21_genome'],
                  ['31_nearTSS'],
                  ['21_genome'],
                  ['31_2_nearTSS'],
                  ['31_3_nearTSS']]

#for each gene/TSS, go through each sgRNA in descending order of predicted score
#if an sgRNA passes the restriction site, overlap, and off-target filters, accept it into the library
#if the number of sgRNAs accepted is less than sgRNAsToPick, reduce off-target stringency by one and continue
v2Groups = v2Table.groupby([('library table v2','gene'),('library table v2','transcripts')])
newSgIds = []
unfinishedTss = []
for (gene, transcript), group in v2Groups:
    geneSgIds = []
    geneLeftPositions = []
    empiricalSgIds = dict()
    
    stringency = 0
    
    while len(geneSgIds) < sgRNAsToPick and stringency < len(offTargetLevels):
        for sgId_v2, row in group.sort_values(('predicted score','predicted score'), ascending=False).iterrows():
            oligoSeq = upstreamConstant + row[('library table v2','sequence')] + downstreamConstant
            leftPos = row[('sgRNA info', 'position')] - (23 if row[('sgRNA info', 'strand')] == '-' else 0)
            if len(geneSgIds) < sgRNAsToPick and row['off-target filters'].loc[offTargetLevels[stringency]].all() \
                and matchREsites(oligoSeq, restrictionSites) \
                and checkOverlaps(leftPos, geneLeftPositions, nonoverlapMin):
                geneSgIds.append((sgId_v2,
                                  gene,transcript,
                                  row[('library table v2','sequence')], oligoSeq,
                                  row[('predicted score','predicted score')], np.nan,
                                 stringency))
                geneLeftPositions.append(leftPos)
                
        stringency += 1
            
    if len(geneSgIds) < sgRNAsToPick:
        unfinishedTss.append((gene, transcript)) #if the number of accepted sgRNAs is still less than sgRNAsToPick, discard gene
    else:
        newSgIds.extend(geneSgIds)
        
libraryTable_complete = pd.DataFrame(newSgIds, columns = ['sgID', 'gene', 'transcript','protospacer sequence', 'oligo sequence',
 'predicted score', 'empirical score', 'off-target stringency']).set_index('sgID')

In [34]:
print len(libraryTable_complete)

80


In [35]:
#number of sgRNAs accepted at each stringency level
libraryTable_complete.groupby('off-target stringency').agg(len).iloc[:,0]

off-target stringency
0    80
Name: gene, dtype: int64

In [36]:
#number of TSSs with fewer than required number of sgRNAs (and thus not included in the library)
print len(unfinishedTss)

0


In [37]:
libraryTable_complete.head()

,gene,transcript,protospacer sequence,oligo sequence,predicted score,empirical score,off-target stringency
sgID,,,,,,,
AADACL2_+_151451732.23-P1P2,AADACL2,P1P2,GGTAGAGCAATTGTAGACTT,CCACCTTGTTGGGTAGAGCAATTGTAGACTTGTTTAAGAGCTAAGCTG,0.693926,NaN,0
AADACL2_-_151452019.23-P1P2,AADACL2,P1P2,GATGACTTATTGACTAAAAA,CCACCTTGTTGGATGACTTATTGACTAAAAAGTTTAAGAGCTAAGCTG,0.451392,NaN,0
AADACL2_+_151452121.23-P1P2,AADACL2,P1P2,GACTGTTACTCACAGATATA,CCACCTTGTTGGACTGTTACTCACAGATATAGTTTAAGAGCTAAGCTG,0.426695,NaN,0
AADACL2_-_151451828.23-P1P2,AADACL2,P1P2,GTGGAAAAAGGGATATTATG,CCACCTTGTTGGTGGAAAAAGGGATATTATGGTTTAAGAGCTAAGCTG,0.404655,NaN,0
AADACL2_-_151451931.23-P1P2,AADACL2,P1P2,GAGCTGGAAAATAATGGCCT,CCACCTTGTTGGAGCTGGAAAATAATGGCCTGTTTAAGAGCTAAGCTG,0.404269,NaN,0


# 5. Design negative controls matching the base composition of the library

In [38]:
#calcluate the base frequency at each position of the sgRNA, then generate random sequences weighted by this frequency
def getBaseFrequencies(libraryTable, baseConversion = {'G':0, 'C':1, 'T':2, 'A':3}):
    baseArray = np.zeros((len(libraryTable),20))

    for i, (index, seq) in enumerate(libraryTable['protospacer sequence'].iteritems()):
        for j, char in enumerate(seq.upper()):
            baseArray[i,j] = baseConversion[char]

    baseTable = pd.DataFrame(baseArray, index = libraryTable.index)
    
    baseFrequencies = baseTable.apply(lambda col: col.groupby(col).agg(len)).fillna(0) / len(baseTable)
    baseFrequencies.index = ['G','C','T','A']
    
    baseCumulativeFrequencies = baseFrequencies.copy()
    baseCumulativeFrequencies.loc['C'] = baseFrequencies.loc['G'] + baseFrequencies.loc['C']
    baseCumulativeFrequencies.loc['T'] = baseFrequencies.loc['G'] + baseFrequencies.loc['C'] + baseFrequencies.loc['T']
    baseCumulativeFrequencies.loc['A'] = baseFrequencies.loc['G'] + baseFrequencies.loc['C'] + baseFrequencies.loc['T'] + baseFrequencies.loc['A']

    return baseFrequencies, baseCumulativeFrequencies

def generateRandomSequence(baseCumulativeFrequencies):
    randArray = np.random.random(baseCumulativeFrequencies.shape[1])
    
    seq = []
    for i, col in baseCumulativeFrequencies.iteritems():
        for base, freq in col.iteritems():
            if randArray[i] < freq:
                seq.append(base)
                break
                
    return ''.join(seq)

In [40]:
baseCumulativeFrequencies = getBaseFrequencies(libraryTable_complete)[1]
negList = []
numberToGenerate = 1000 #can generate many more; some will be filtered out by off-targets, and you can always select an arbitrary subset for inclusion into the library
for i in range(numberToGenerate):
    negList.append(generateRandomSequence(baseCumulativeFrequencies))
negTable = pd.DataFrame(negList, index=['non-targeting_' + str(i) for i in range(numberToGenerate)], columns = ['sequence'])

fqFile = 'temp_bowtie_files/bowtie_input_negs.fq'
outputTempBowtieFastq(negTable, fqFile)

In [41]:
#similar to targeting sgRNA off-target scoring, but looking for sgRNAs with 0 alignments
alignmentList = [(31,1,'~/indices/hg19.ensemblTSSflank500b','31_nearTSS_negs'),
                (21,1,'~/indices/hg19_maskChrMandPAR','21_genome_negs')]

alignmentColumns = []
for btThreshold, mflag, bowtieIndex, runname in alignmentList:

    alignedFile = 'temp_bowtie_files/' + runname + '_aligned.txt'
    unalignedFile = 'temp_bowtie_files/' + runname + '_unaligned.fq'
    maxFile = 'temp_bowtie_files/' + runname + '_max.fq'
    
    bowtieString = 'bowtie -n 3 -l 15 -e '+str(btThreshold)+' -m ' + str(mflag) + ' --nomaqround -a --tryhard -p 16 --chunkmbs 256 ' + bowtieIndex + ' --suppress 5,6,7 --un ' + unalignedFile + ' --max ' + maxFile + ' '+ ' -q '+fqFile+' '+ alignedFile
    print bowtieString
    print subprocess.call(bowtieString, shell=True)

    #read unaligned file for negs, and then don't flip boolean of alignmentTable
    with open(unalignedFile) as infile:
        sgsAligning = set()
        for i, line in enumerate(infile):
            if i%4 == 0: #id line
                sgsAligning.add(line.strip()[1:])

    alignmentColumns.append(negTable.apply(lambda row: row.name in sgsAligning, axis=1))
    
alignmentTable = pd.concat(alignmentColumns,axis=1, keys=zip(*alignmentList)[3])
alignmentTable.head()

bowtie -n 3 -l 15 -e 31 -m 1 --nomaqround -a --tryhard -p 16 --chunkmbs 256 ~/indices/hg19.ensemblTSSflank500b --suppress 5,6,7 --un temp_bowtie_files/31_nearTSS_negs_unaligned.fq --max temp_bowtie_files/31_nearTSS_negs_max.fq  -q temp_bowtie_files/bowtie_input_negs.fq temp_bowtie_files/31_nearTSS_negs_aligned.txt
0
bowtie -n 3 -l 15 -e 21 -m 1 --nomaqround -a --tryhard -p 16 --chunkmbs 256 ~/indices/hg19_maskChrMandPAR --suppress 5,6,7 --un temp_bowtie_files/21_genome_negs_unaligned.fq --max temp_bowtie_files/21_genome_negs_max.fq  -q temp_bowtie_files/bowtie_input_negs.fq temp_bowtie_files/21_genome_negs_aligned.txt
0


,31_nearTSS_negs,21_genome_negs
non-targeting_0,True,True
non-targeting_1,True,True
non-targeting_2,True,True
non-targeting_3,True,False
non-targeting_4,True,True


In [42]:
acceptedNegList = []
negCount = 0
for i, (name, row) in enumerate(pd.concat((negTable,alignmentTable),axis=1, keys=['seq','alignment']).iterrows()):
    oligo = upstreamConstant + row['seq','sequence'] + downstreamConstant
    if row['alignment'].all() and matchREsites(oligo, restrictionSites):
        acceptedNegList.append(('non-targeting_%05d' % negCount, 'negative_control', 'na', row['seq','sequence'], oligo, 0))
        negCount += 1
        
acceptedNegs = pd.DataFrame(acceptedNegList, columns = ['sgId', 'gene', 'transcript', 'protospacer sequence', 'oligo sequence', 'off-target stringency']).set_index('sgId')

In [43]:
acceptedNegs.head()

,gene,transcript,protospacer sequence,oligo sequence,off-target stringency
sgId,,,,,
non-targeting_00000,negative_control,na,GGTTTCGCGCGCTTACAGAT,CCACCTTGTTGGGTTTCGCGCGCTTACAGATGTTTAAGAGCTAAGCTG,0
non-targeting_00001,negative_control,na,GGTGGTCGAAGATAGCGAGC,CCACCTTGTTGGGTGGTCGAAGATAGCGAGCGTTTAAGAGCTAAGCTG,0
non-targeting_00002,negative_control,na,GCTCTTGACAAATTCAAGCT,CCACCTTGTTGGCTCTTGACAAATTCAAGCTGTTTAAGAGCTAAGCTG,0
non-targeting_00003,negative_control,na,GGCCGGGAGAGCGGGAACTC,CCACCTTGTTGGGCCGGGAGAGCGGGAACTCGTTTAAGAGCTAAGCTG,0
non-targeting_00004,negative_control,na,GTCGCAAGCCGGGGTAGGGT,CCACCTTGTTGGTCGCAAGCCGGGGTAGGGTGTTTAAGAGCTAAGCTG,0


In [ ]:
libraryTable_complete.to_csv(LIBRARY_WITHOUT_NEGATIVES_PATH, sep='\t')
acceptedNegs.to_csv(NEGATIVE_CONTROLS_PATH,sep='\t')

# 6. Finalizing library design

* divide genes into sublibrary groups (if required)
* assign negative control sgRNAs to sublibrary groups; ~1-2% of the number of sgRNAs in the library is a good rule-of-thumb
* append PCR adapter sequences (~18bp) to each end of the oligo sequences to enable amplification of the oligo pool; each sublibary should have an orthogonal sequence so they can be cloned separately